## Trajectory analysis template
General geometric trajectory analysis from a ```.mymridon``` experiment file to be saved as a csv, which can be further analyzed in the optional part or exported into other software such as RStudio. Operations like done here are typically heavy and python benefits from the effeciency of the underlying ```myrmidon``` library to a big extent, especially in combination with the ```numpy``` math library.
This notebook is a walk trough a sample usage of the following:
* the `py-fort-myrmidon` library ([Documentation](https://formicidae-tracker.github.io/myrmidon/latest/))
* numpy quick start [tutorial](https://numpy.org/devdocs/user/quickstart.html)

In [ ]:
import py_fort_myrmidon as fm
import numpy as np  # Fundamental math library in python. Here used only for convience: to save the csv.
from datetime import datetime, timedelta  # For convenient handling of time and date
import numpy as np  # Basic math library
import pandas as pd  # Used to create a dataframe, similar to the structure used in R
import matplotlib.pyplot as plt  # Optional: for plorring
import gc

# Optional: makes plots interactive:
# %matplotlib widget

# %load_ext blackcellmagic

## Calculate displacement in specific phase

In [ ]:
def calculate_displacement(trajectory):
    """Function to create a dataframe containing displacment per detection in a trajectory

    Args:
        trajectory (fort-myrmidon Ant trajectory): A trajectory object obtained using the fort-myrmidon API

    Returns:
        pd.DataFrame: Outputs a pandas dataframe with the time stamps as index, AntID, X and Y coordinates, displacment and time difference for the timestamp (relative to the preceding timestamp)
    """
    # calculate displacement and time difference between detections
    dxdy = np.diff(
        trajectory.Positions[:, 1:3], axis=0
    )  # x-y differerence between detections (2d vector)
    ds = np.linalg.norm(dxdy, axis=1)  # displacement between detections (1d vector)
    ds = np.insert(ds, 0, 0)  # Add a zero displacement for the first detection
    dt = np.diff(
        trajectory.Positions[:, 0]
    )  # Time interval between detections (1d vector)
    # Get time information
    dt = np.insert(dt, 0, 0)  # Add a zero time difference for the first detection
    trajectory_start = trajectory.Start.ToDateTime()  # Get starting time of trajectory
    trajectory_time = trajectory.Positions[
        :, 0
    ]  # get time difference of detections from starting time
    trajectory_time = pd.to_timedelta(
        trajectory_time, unit="s"
    )  # Convert time difference to timedelta
    time_stamps = trajectory_start + trajectory_time  # Get time stamps
    # time_stamps = time_stamps.delete(
    #     0
    # )  # Remove first time stamp, since we don't have displacement for it
    # Create dataframe
    disp_df = pd.DataFrame(index=time_stamps, data=ds, columns=["displacement"])
    disp_df["time_diff"] = dt
    disp_df["AntID"] = trajectory.Ant  # Add Ant ID
    disp_df["X"] = trajectory.Positions[:, 1]  # Add X position
    disp_df["Y"] = trajectory.Positions[:, 2]  # Add Y position
    disp_df["Space"] = trajectory.Space  # Add Space ID
    disp_df = disp_df[["AntID", "Space", "X", "Y", "displacement", "time_diff"]]
    return disp_df

In [ ]:
def obtain_displacement_df(exp, start_time, end_time, colony, phase, *time_resolution):
    """Output a csv the displacement (or mean displacement) of each ant in the experiment between t_start and t_end.

    Args:
        exp (fm.Experiment): the experiment object
        t_start (fm.Time): the start time
        t_end (fm.Time): the end time
        phase (fm.Phase): the phase to consider
        time_resolution (string): the time resolution to consider, given in the format used by pandas grouper
    """
    t_start_fm = fm.Time(start_time)
    t_end_fm = fm.Time(end_time)
    freq = "".join(time_resolution)
    trajectories = fm.Query.ComputeAntTrajectories(
        experiment=exp,
        start=t_start_fm,
        end=t_end_fm,
        maximumGap=1000 * fm.Duration.Hour,
    )
    disp_df_list = [calculate_displacement(t) for t in trajectories]
    disp_df = pd.concat(disp_df_list)
    disp_df.index.name = "time"
    # Since thee are instancs where the ant may not be detected for a given duration, we filter these out.
    # We keep the filter to a value of 1s. This ensures that if there is a gap in dtection greater than a second,
    # that specific value is not included in the calculation
    disp_df = disp_df[
        disp_df["time_diff"] < 1
    ]  # Filter out rows where time_diff is greater than 1 second

    # Define a standard datetime format for CSV output
    csv_date_format = "%Y-%m-%d %H:%M:%S"

    day_exp = start_time.strftime("%Y%m%d")
    hr_strt = start_time.strftime("%H%M")
    hr_end = end_time.strftime("%H%M")
    if time_resolution:
        disp_df = (
            disp_df.groupby([pd.Grouper(freq=freq), "AntID", "Space"])
            .agg(
                {"X": "mean", "Y": "mean", "displacement": "mean", "time_diff": "mean"}
            )
            .reset_index()
        )
        df_fn = f"{'MeanDisplacement_'}{freq}{'_'}{colony}{'_'}{phase}{'_'}{day_exp}{'_'}{hr_strt}{'-'}{hr_end}{'.csv'}"
        disp_df.to_csv(df_fn, index=False, date_format=csv_date_format)
    else:
        df_fn = f"{'Displacement_'}{exp.Name}{'_'}{colony}{'_'}{day_exp}{'_'}{hr_strt}{'-'}{hr_end}{'.csv'}"
        disp_df.reset_index().to_csv(df_fn, index=False, date_format=csv_date_format)

    # Remove files
    del disp_df
    del trajectories
    del disp_df_list

    gc.collect()  # Call garbage collector to free up memory

In [ ]:
phase_list = [
    "Control",
    "R1",
    "R2",
    "R3",
    "R4",
    "R5",
    "PostC",
    "PostR1",
    "PostR2",
    "PostR3",
    "PostR4",
    "PostR5",
]
time_resolution = "1s"
# phase_list = phase_list[1:6]
# phase_list = [phase_name + 'Plus2' for phase_name in phase_list]

### Colony Cfel 42

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-2/Woundcare Experiment1/Cfell_wound_col42.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
colony = "Cfel42"
phase_starts_exp = [
    datetime(2022, 5, 1, 15, 54).astimezone(tz=None),
    datetime(2022, 5, 2, 16, 3).astimezone(tz=None),
    datetime(2022, 5, 3, 15, 53).astimezone(tz=None),
    datetime(2022, 5, 4, 15, 50).astimezone(tz=None),
    datetime(2022, 5, 5, 15, 50).astimezone(tz=None),
    datetime(2022, 5, 6, 15, 55).astimezone(tz=None),
]
phase_starts_post = [
    datetime(2022, 5, 2, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 3, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 4, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 5, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 6, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 7, 9, 0).astimezone(tz=None),
]
phase_starts = phase_starts_exp + phase_starts_post
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
# Run only if you need to get displacement for 2 hours after estart of each treatment phase
phase_starts = phase_starts[1:6]
phase_starts = [(start_time + timedelta(hours=2)) for start_time in phase_starts]
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
cfel42_disp = [
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for start, end, phase in zip(phase_starts, phase_ends, phase_list)
]

### Colony Cfel1

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-2/Woundcare Experiment2/woundcare_cfell1_T2.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
colony = "Cfel1"
phase_starts_exp = [
    datetime(2022, 6, 4, 14, 48).astimezone(tz=None),
    datetime(2022, 6, 5, 14, 57).astimezone(tz=None),
    datetime(2022, 6, 6, 14, 30).astimezone(tz=None),
    datetime(2022, 6, 7, 14, 49).astimezone(tz=None),
    datetime(2022, 6, 8, 14, 43).astimezone(tz=None),
    datetime(2022, 6, 9, 15, 5).astimezone(tz=None),
]
phase_starts_post = [
    datetime(2022, 6, 5, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 6, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 7, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 8, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 9, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 10, 8, 0).astimezone(tz=None),
]
phase_starts = phase_starts_exp + phase_starts_post
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
# Run only if you need to get displacement for 2 hours after estart of each treatment phase
phase_starts = phase_starts[1:6]
phase_starts = [(start_time + timedelta(hours=2)) for start_time in phase_starts]
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
cfel1_disp = [
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for start, end, phase in zip(phase_starts, phase_ends, phase_list)
]

### Colony Cfel54

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-2/Woundcare Experiment3/woundcare_cfell54_T3.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
colony = "Cfel54"
phase_starts_exp = [
    datetime(2022, 6, 19, 14, 26).astimezone(tz=None),
    datetime(2022, 6, 20, 14, 35).astimezone(tz=None),
    datetime(2022, 6, 21, 14, 21).astimezone(tz=None),
    datetime(2022, 6, 22, 14, 28).astimezone(tz=None),
    datetime(2022, 6, 23, 14, 14).astimezone(tz=None),
    datetime(2022, 6, 24, 14, 31).astimezone(tz=None),
]
phase_starts_post = [
    datetime(2022, 6, 20, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 21, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 22, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 23, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 24, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 25, 8, 0).astimezone(tz=None),
]
phase_starts = phase_starts_exp + phase_starts_post
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
# Run only if you need to get displacement for 2 hours after estart of each treatment phase
phase_starts = phase_starts[1:6]
phase_starts = [(start_time + timedelta(hours=2)) for start_time in phase_starts]
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
cfel54_disp = [
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for start, end, phase in zip(phase_starts, phase_ends, phase_list)
]

### Colony Cfel55

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-3/InfectionExp_Cfel55/InfectionExpCol55.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
colony = "Cfel55"
phase_starts_exp = [
    datetime(2023, 4, 18, 14, 40).astimezone(tz=None),
    datetime(2023, 4, 20, 15, 45).astimezone(tz=None),
    datetime(2023, 4, 21, 14, 48).astimezone(tz=None),
    datetime(2023, 4, 22, 14, 17).astimezone(tz=None),
    datetime(2023, 4, 23, 14, 0).astimezone(tz=None),
    datetime(2023, 4, 24, 14, 54).astimezone(tz=None),
]
phase_starts_post = [
    datetime(2023, 4, 20, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 21, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 22, 7, 30).astimezone(tz=None),
    datetime(2023, 4, 23, 7, 30).astimezone(tz=None),
    datetime(2023, 4, 24, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 25, 8, 0).astimezone(tz=None),
]
phase_starts = phase_starts_exp + phase_starts_post
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
# Run only if you need to get displacement for 2 hours after estart of each treatment phase
phase_starts = phase_starts[1:6]
phase_starts = [(start_time + timedelta(hours=2)) for start_time in phase_starts]
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
cfel55_disp = [
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for start, end, phase in zip(phase_starts, phase_ends, phase_list)
]

In [ ]:
obtain_displacement_df(
    exp,
    datetime(2023, 4, 14, 0, 1).astimezone(tz=None),
    datetime(2023, 4, 18, 23, 59).astimezone(tz=None),
    "Baseline",
    "1m",
)

In [ ]:
baseline_days = ["Baseline1", "Baseline2", "Baseline3", "Baseline4", "Baseline5"]
exp_start = datetime(2023, 4, 14, 0, 1).astimezone(tz=None)
baseline_starts = [
    exp_start,
    exp_start + timedelta(days=1),
    exp_start + timedelta(days=2),
    exp_start + timedelta(days=3),
    exp_start + timedelta(days=4),
]
baseline_ends = [
    start_time + timedelta(hours=22, minutes=59) for start_time in baseline_starts
]

In [ ]:
cfel55_baseline_disp = [
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for start, end, phase in zip(baseline_starts, baseline_ends, baseline_days)
]

In [ ]:
start_time = datetime(2023, 4, 14, 0, 1).astimezone(tz=None)
end_time = datetime(2023, 4, 14, 23, 59).astimezone(tz=None)
t_start = fm.Time(start_time)
t_end = fm.Time(end_time)
freq = "".join("1m")

In [ ]:
trajectories = fm.Query.ComputeAntTrajectories(
    experiment=exp, start=t_start, end=t_end, maximumGap=1000 * fm.Duration.Hour
)

In [ ]:
import dask.dataframe as dd


def calculate_displacement_dask(trajectory):
    """Function to create a dataframe containing displacment per detection in a trajectory

    Args:
        trajectory (fort-myrmidon Ant trajectory): A trajectory object obtained using the fort-myrmidon API

    Returns:
        pd.DataFrame: Outputs a pandas dataframe with the time stamps as index, AntID, X and Y coordinates, displacment and time difference for the timestamp (relative to the preceding timestamp)
    """
    # calculate displacement and time difference between detections
    dxdy = np.diff(
        trajectory.Positions[:, 1:3], axis=0
    )  # x-y differerence between detections (2d vector)
    ds = np.linalg.norm(dxdy, axis=1)  # displacement between detections (1d vector)
    ds = np.insert(ds, 0, 0)  # Add a zero displacement for the first detection
    dt = np.diff(
        trajectory.Positions[:, 0]
    )  # Time interval between detections (1d vector)
    # Get time information
    dt = np.insert(dt, 0, 0)  # Add a zero time difference for the first detection
    trajectory_start = trajectory.Start.ToDateTime()  # Get starting time of trajectory
    trajectory_time = trajectory.Positions[
        :, 0
    ]  # get time difference of detections from starting time
    trajectory_time = pd.to_timedelta(
        trajectory_time, unit="s"
    )  # Convert time difference to timedelta
    time_stamps = trajectory_start + trajectory_time  # Get time stamps
    # time_stamps = time_stamps.delete(
    #     0
    # )  # Remove first time stamp, since we don't have displacement for it
    # Create dataframe
    disp_df = pd.DataFrame(data=ds, columns=["displacement"])
    disp_df["time"] = time_stamps
    disp_df["time_diff"] = dt
    disp_df["AntID"] = trajectory.Ant  # Add Ant ID
    disp_df["X"] = trajectory.Positions[:, 1]  # Add X position
    disp_df["Y"] = trajectory.Positions[:, 2]  # Add Y position
    disp_df["Space"] = trajectory.Space  # Add Space ID
    disp_df = disp_df[["time", "AntID", "Space", "X", "Y", "displacement", "time_diff"]]
    return disp_df

In [ ]:
disp_df_list = [calculate_displacement_dask(t) for t in trajectories]

In [ ]:
# Assume disp_df_list is a list of pandas DataFrames
disp_df = dd.concat([dd.from_pandas(df, npartitions=1) for df in disp_df_list])
disp_df

In [ ]:
disp_df = disp_df.set_index("time")
# disp_df = disp_df.repartition(divisions=disp_df.divisions)

# Filter out rows where time_diff is greater than 1 second
disp_df = disp_df[disp_df["time_diff"] < 1]

In [ ]:
disp_df_sub = (
    disp_df.resample(freq)
    .agg({"X": "mean", "Y": "mean", "displacement": "mean", "time_diff": "mean"})
    .reset_index()
)

In [ ]:
disp_df.head(5)

In [ ]:
day_exp = start_time.strftime("%Y%m%d")
hr_strt = start_time.strftime("%H%M")
hr_end = end_time.strftime("%H%M")
df_fn = f"{'MeanDisplacement_'}{freq}{'_'}{colony}{'_'}{'Baseline'}{'_'}{day_exp}{'_'}{hr_strt}{'-'}{hr_end}{'.csv'}"
disp_df.compute().to_csv(df_fn, index=False)

### Colony Cfel13

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-3/InfectionExp_Cfel13/InfectionExp_Cfel13.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
colony = "Cfel13"
phase_starts_exp = [
    datetime(2023, 4, 23, 15, 5).astimezone(tz=None),
    datetime(2023, 4, 24, 15, 29).astimezone(tz=None),
    datetime(2023, 4, 25, 14, 19).astimezone(tz=None),
    datetime(2023, 4, 26, 15, 3).astimezone(tz=None),
    datetime(2023, 4, 27, 16, 43).astimezone(tz=None),
    datetime(2023, 4, 28, 14, 27).astimezone(tz=None),
]
phase_starts_post = [
    datetime(2023, 4, 24, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 25, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 26, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 27, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 28, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 29, 8, 0).astimezone(tz=None),
]
phase_starts = phase_starts_exp + phase_starts_post
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
# Run only if you need to get displacement for 2 hours after estart of each treatment phase
phase_starts = phase_starts[1:6]
phase_starts = [(start_time + timedelta(hours=2)) for start_time in phase_starts]
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
cfel13_disp = [
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for start, end, phase in zip(phase_starts, phase_ends, phase_list)
]

### Colony Cfel64

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-1/InfectionExp_Cfel64/InfectionExpCol64.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
colony = "Cfel64"
phase_starts_exp = [
    datetime(2023, 5, 31, 15, 5).astimezone(tz=None),
    datetime(2023, 6, 1, 15, 51).astimezone(tz=None),
    datetime(2023, 6, 2, 14, 44).astimezone(tz=None),
    datetime(2023, 6, 3, 14, 50).astimezone(tz=None),
    datetime(2023, 6, 4, 14, 43).astimezone(tz=None),
    datetime(2023, 6, 5, 14, 52).astimezone(tz=None),
]
phase_starts_post = [
    datetime(2023, 6, 1, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 2, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 3, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 4, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 5, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 6, 8, 0).astimezone(tz=None),
]
phase_starts = phase_starts_exp + phase_starts_post
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
phase_starts = phase_starts[1:6]
phase_starts = [(start_time + timedelta(hours=2)) for start_time in phase_starts]
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
cfel64_disp = [
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for start, end, phase in zip(phase_starts, phase_ends, phase_list)
]

# Baseline 6 hour phases

For the baseline dataset outputting the raw trajectories 

In [ ]:
# Create list of colonies repeated 5 times for each colony
# colony_list = ["Cfel42", "Cfel1", "Cfel54", "Cfel13", "Cfel55", "Cfel64"]
colony_list = ["Cfel42", "Cfel1", "Cfel54", "Cfel13", "Cfel55"]
# Multiple each element 5 times
colony_list = [x for x in colony_list for _ in range(5)]
# Time resoltion
time_resolution = "1s"

In [ ]:
phase_list = ["Base1", "Base2", "Base3", "Base4", "Base5"] * 5
# Change to 6 if all 6 colonies are used

In [ ]:
# Myrmidon experiment object list
cfel42_exp = fm.Experiment.Open(
    "/media/ebiag/Ebi-2/Woundcare Experiment1/Cfell_wound_col42.myrmidon"
)
cfel1_exp = fm.Experiment.Open(
    "/media/ebiag/Ebi-2/Woundcare Experiment2/woundcare_cfell1_T2.myrmidon"
)
cfel54_exp = fm.Experiment.Open(
    "/media/ebiag/Ebi-2/Woundcare Experiment3/woundcare_cfell54_T3.myrmidon"
)
cfel13_exp = fm.Experiment.Open(
    "/media/ebiag/Ebi-3/InfectionExp_Cfel13/InfectionExp_Cfel13.myrmidon"
)
cfel55_exp = fm.Experiment.Open(
    "/media/ebiag/Ebi-1/InfectionExp_Cfel55/InfectionExpCol55.myrmidon"
)
# cfel64_exp = fm.Experiment.Open(
#     "/media/ebiag/Ebi-4/InfectionExp_Cfel64/InfectionExpCol64.myrmidon"
# )

exp_list = [cfel42_exp, cfel1_exp, cfel54_exp, cfel13_exp, cfel55_exp]
# Repeat each element 5 times
exp_list = [x for x in exp_list for _ in range(5)]

In [ ]:
# Start times for experimental phase
exp_start_times = [
    # Cfel 42
    ## Baseline phases
    datetime(2022, 4, 27, 15, 54).astimezone(tz=None),
    datetime(2022, 4, 28, 15, 54).astimezone(tz=None),
    datetime(2022, 4, 29, 15, 54).astimezone(tz=None),
    datetime(2022, 4, 30, 15, 54).astimezone(tz=None),
    datetime(2022, 5, 1, 15, 54).astimezone(tz=None),
    # Cfel 1
    ## Baseline phases
    datetime(2022, 5, 31, 14, 48).astimezone(tz=None),
    datetime(2022, 6, 1, 14, 48).astimezone(tz=None),
    datetime(2022, 6, 2, 14, 48).astimezone(tz=None),
    # datetime(2022, 6, 3, 14, 48).astimezone(tz=None), # Data corruption on this day
    datetime(2022, 6, 4, 14, 48).astimezone(tz=None),
    # Cfel 54
    ## Baseline phases
    datetime(2022, 6, 15, 14, 26).astimezone(tz=None),
    datetime(2022, 6, 16, 14, 26).astimezone(tz=None),
    datetime(2022, 6, 17, 14, 26).astimezone(tz=None),
    datetime(2022, 6, 18, 14, 26).astimezone(tz=None),
    datetime(2022, 6, 19, 14, 26).astimezone(tz=None),
    # Cfel 13
    ## Baseline phases
    datetime(2023, 4, 19, 15, 5).astimezone(tz=None),
    datetime(2023, 4, 20, 15, 5).astimezone(tz=None),
    datetime(2023, 4, 21, 15, 5).astimezone(tz=None),
    datetime(2023, 4, 22, 15, 5).astimezone(tz=None),
    datetime(2023, 4, 23, 15, 5).astimezone(tz=None),
    # Cfel 55
    ## Baseline phases
    datetime(2023, 4, 14, 14, 40).astimezone(tz=None),
    datetime(2023, 4, 15, 14, 40).astimezone(tz=None),
    datetime(2023, 4, 16, 14, 40).astimezone(tz=None),
    datetime(2023, 4, 17, 14, 40).astimezone(tz=None),
    datetime(2023, 4, 18, 14, 40).astimezone(tz=None),
    # # Cfel 64
    # ## Baseline phases
    # datetime(2023, 5, 27, 15, 5).astimezone(tz=None),
    # datetime(2023, 5, 28, 15, 5).astimezone(tz=None),
    # datetime(2023, 5, 29, 15, 5).astimezone(tz=None),
    # datetime(2023, 5, 30, 15, 5).astimezone(tz=None),
    # datetime(2023, 5, 31, 15, 5).astimezone(tz=None),
]
exp_end_times = [(start_time + timedelta(hours=6)) for start_time in exp_start_times]

In [ ]:
# Remove Base4 from Cfel1 as we don't have data on that day due to data corruption
# It is the at index 8
del colony_list[8]
del exp_list[8]
del phase_list[8]

In [ ]:
time_resolution = "1s"
phase_list = [
    "Base1",
    "Base2",
    "Base3",
    "Base4",
    "Base5",
]

In [ ]:
exp = fm.Experiment.Open(
    "/media/egeorge/Ebi-2/Woundcare Experiment1/Cfell_wound_col42.myrmidon"
)
colony = "Cfel42"
exp_start_times = [
    datetime(2022, 4, 27, 15, 54).astimezone(tz=None),
    datetime(2022, 4, 28, 15, 54).astimezone(tz=None),
    datetime(2022, 4, 29, 15, 54).astimezone(tz=None),
    datetime(2022, 4, 30, 15, 54).astimezone(tz=None),
    datetime(2022, 5, 1, 15, 54).astimezone(tz=None),
]
exp_end_times = [(start_time + timedelta(hours=6)) for start_time in exp_start_times]
[
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for start, end, phase in zip(
        exp_start_times,
        exp_end_times,
        phase_list,
    )
]

In [ ]:
exp = fm.Experiment.Open(
    "/media/egeorge/Ebi-2/Woundcare Experiment2/woundcare_cfell1_T2.myrmidon"
)
colony = "Cfel1"
exp_start_times = [
    datetime(2022, 5, 31, 14, 48).astimezone(tz=None),
    datetime(2022, 6, 1, 14, 48).astimezone(tz=None),
    datetime(2022, 6, 2, 14, 48).astimezone(tz=None),
    # datetime(2022, 6, 3, 14, 48).astimezone(tz=None), # Data corruption on this day
    datetime(2022, 6, 4, 14, 48).astimezone(tz=None),
]
exp_end_times = [(start_time + timedelta(hours=6)) for start_time in exp_start_times]
# Modified phase list
phase_list_mod = [
    "Base1",
    "Base2",
    "Base3",
    "Base5",
]
[
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for start, end, phase in zip(
        exp_start_times,
        exp_end_times,
        phase_list_mod,
    )
]

In [ ]:
exp = fm.Experiment.Open(
    "/media/egeorge/Ebi-2/Woundcare Experiment3/woundcare_cfell54_T3.myrmidon"
)
colony = "Cfel54"
exp_start_times = [
    datetime(2022, 6, 15, 14, 26).astimezone(tz=None),
    datetime(2022, 6, 16, 14, 26).astimezone(tz=None),
    datetime(2022, 6, 17, 14, 26).astimezone(tz=None),
    datetime(2022, 6, 18, 14, 26).astimezone(tz=None),
    datetime(2022, 6, 19, 14, 26).astimezone(tz=None),
]
exp_end_times = [(start_time + timedelta(hours=6)) for start_time in exp_start_times]

In [ ]:
[
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for start, end, phase in zip(
        exp_start_times,
        exp_end_times,
        phase_list,
    )
]

In [ ]:
obtain_displacement_df(
    exp,
    datetime(2022, 6, 15, 14, 26).astimezone(tz=None),
    datetime(2022, 6, 15, 20, 26).astimezone(tz=None),
    colony,
    "Base1",
    time_resolution,
)

In [ ]:
exp = fm.Experiment.Open(
    "/media/egeorge/Ebi-4/InfectionExp_Cfel13/InfectionExp_Cfel13.myrmidon"
)
colony = "Cfel13"
exp_start_times = [
    datetime(2023, 4, 19, 15, 5).astimezone(tz=None),
    datetime(2023, 4, 20, 15, 5).astimezone(tz=None),
    datetime(2023, 4, 21, 15, 5).astimezone(tz=None),
    datetime(2023, 4, 22, 15, 5).astimezone(tz=None),
    datetime(2023, 4, 23, 15, 5).astimezone(tz=None),
]
exp_end_times = [(start_time + timedelta(hours=6)) for start_time in exp_start_times]
[
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for start, end, phase in zip(
        exp_start_times,
        exp_end_times,
        phase_list,
    )
]

In [ ]:
exp = fm.Experiment.Open(
    "/media/ebiag/Ebi-1/InfectionExp_Cfel55/InfectionExpCol55.myrmidon"
)
colony = "Cfel55"
exp_start_times = [
    datetime(2023, 4, 14, 14, 40).astimezone(tz=None),
    datetime(2023, 4, 15, 14, 40).astimezone(tz=None),
    datetime(2023, 4, 16, 14, 40).astimezone(tz=None),
    datetime(2023, 4, 17, 14, 40).astimezone(tz=None),
    datetime(2023, 4, 18, 14, 40).astimezone(tz=None),
]
exp_end_times = [(start_time + timedelta(hours=6)) for start_time in exp_start_times]
[
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for start, end, phase in zip(
        exp_start_times,
        exp_end_times,
        phase_list,
    )
]


In [ ]:
exp = fm.Experiment.Open(
    "/media/egeorge/Ebi-4/InfectionExp_Cfel64/InfectionExpCol64.myrmidon"
)
colony = "Cfel64"
exp_start_times = [
    datetime(2023, 5, 27, 15, 5).astimezone(tz=None),
    datetime(2023, 5, 28, 15, 5).astimezone(tz=None),
    datetime(2023, 5, 29, 15, 5).astimezone(tz=None),
    datetime(2023, 5, 30, 15, 5).astimezone(tz=None),
    datetime(2023, 5, 31, 15, 5).astimezone(tz=None),
]
exp_end_times = [(start_time + timedelta(hours=6)) for start_time in exp_start_times]
[
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for start, end, phase in zip(
        exp_start_times,
        exp_end_times,
        phase_list,
    )
]

In [ ]:
[
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for exp, start, end, colony, phase in zip(
        exp_list[5:9],
        exp_start_times[5:9],
        exp_end_times[5:9],
        colony_list[5:9],
        phase_list[5:9],
    )
]

In [ ]:
[
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for exp, start, end, colony, phase in zip(
        exp_list[9:14],
        exp_start_times[9:4],
        exp_end_times[9:14],
        colony_list[9:14],
        phase_list[9:14],
    )
]

In [ ]:
[
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for exp, start, end, colony, phase in zip(
        exp_list[14:19],
        exp_start_times[14:19],
        exp_end_times[14:19],
        colony_list[14:19],
        phase_list[14:19],
    )
]

In [ ]:
[
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for exp, start, end, colony, phase in zip(
        exp_list[19:24],
        exp_start_times[19:24],
        exp_end_times[19:24],
        colony_list[19:24],
        phase_list[19:24],
    )
]

In [ ]:
baseline_df = [
    obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
    for exp, start, end, colony, phase in zip(
        exp_list, exp_start_times, exp_end_times, colony_list, phase_list
    )
]
# baseline_df = [
#     obtain_displacement_df(exp, start, end, colony, phase, time_resolution)
#     for exp, start, end, colony, phase in zip(
#         exp_list[:1],
#         exp_start_times[:1],
#         exp_end_times[:1],
#         colony_list[:1],
#         phase_list[:1],
#     )
# ]

In [ ]:
t_start_fm = fm.Time(datetime(2022, 4, 27, 15, 54).astimezone(tz=None))
t_end_fm = fm.Time(datetime(2022, 4, 27, 21, 54).astimezone(tz=None))
freq = "".join(time_resolution)
print(fm.Time(datetime(2022, 4, 27, 15, 54).astimezone(tz=None)))
start_time_fm = exp_start_times[0]
end_time_fm = exp_end_times[0]
print(fm.Time(start_time))

In [ ]:
trajectories = fm.Query.ComputeAntTrajectories(
    experiment=exp_list[0],
    start=fm.Time(start_time_fm),
    end=fm.Time(end_time_fm),
    maximumGap=1000 * fm.Duration.Hour,
)

In [ ]:
disp_df_list = [calculate_displacement(t) for t in trajectories]
disp_df = pd.concat(disp_df_list)
disp_df.index.name = "time"
disp_df.head(5)

In [ ]:
disp_df = disp_df[disp_df["time_diff"] < 1]

In [ ]:
disp_df = (
    disp_df.groupby([pd.Grouper(freq=freq), "AntID", "Space"])
    .agg({"X": "mean", "Y": "mean", "displacement": "mean", "time_diff": "mean"})
    .reset_index()
)

In [ ]:
disp_df.head(5)

In [ ]:
df_fn = "Trial.csv"
disp_df.to_csv(df_fn, index=False)